In [1]:
#encoding=utf8
import importlib

import urllib.request
from xlrd import open_workbook
from Bio import Entrez
import xml.dom.minidom
import xml.etree.ElementTree as ET
import pandas
import openpyxl

Entrez.email = "lhoang2@illinois.edu"

#Read list of PMIDs from txt file
#CHANGE THIS PATH TO THE FOLDER WHERE YOU STORE THE DATA
PMID_list_file = open("/Data/All_Papers_PMIDs.txt","r")


#Save results to Excel file
from pandas import DataFrame
PMID_list = []
ArtcileTitle_list = []
Abstract_list = []
PublicationType_list = []
MeSH_list = []
PMCID_list = []

for PMID in PMID_list_file.readlines():
    PMID = PMID.replace("\n","")
    print (PMID)

    #Pass PMIDs to PubMed API to get articles' metadata
    link = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=" + PMID + "&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y"            
    f = urllib.request.urlopen(link)   
    print (link)
    tree = ET.parse(f)
    doc = tree.getroot()    

    for child in doc:
        for step_child in child:
            if step_child.tag == "MedlineCitation":
                article_abstract_list = []
                article_publication_type_list = []
                article_MeSH_list = []
                
                article_abstract = "None"
                
                for step_child_level_2 in step_child:
                    if step_child_level_2.tag == "PMID":
                        article_PMID =  step_child_level_2.text
                      
                    if step_child_level_2.tag == "Article":
                        for step_child_level_3 in step_child_level_2:
                            if step_child_level_3.tag == "ArticleTitle":
                                article_title = step_child_level_3.text                         
                            
                            if step_child_level_3.tag == "Abstract":
                                if len(step_child_level_3) >= 1:                                    
                                    for step_child_level_4 in step_child_level_3:
                                        if step_child_level_4.tag == "AbstractText":
                                            article_abstract_list.append(step_child_level_4.text)
                                else:
                                    article_abstract_list.append("None")
                                article_abstract = b" ".join(str(x).encode('utf-8') for x in article_abstract_list)

                            
                            if step_child_level_3.tag == "PublicationTypeList":                                
                                for step_child_level_4 in step_child_level_3:
                                    if step_child_level_4.tag == "PublicationType":
                                        article_publication_type_list.append(step_child_level_4.text)
                                    else:
                                        article_publication_type_list.append("None")
                                                                             
                                article_publication_type = " ;".join(str(x) for x in article_publication_type_list)   
                                                                          
                        PMID_list.append(article_PMID)
                        ArtcileTitle_list.append(article_title)
                        Abstract_list.append(article_abstract)
                        PublicationType_list.append(article_publication_type)
                        
                    if step_child_level_2.tag == "MeshHeadingList":
                        for step_child_level_3 in step_child_level_2:
                            if step_child_level_3.tag == "MeshHeading":
                                for step_child_level_4 in step_child_level_3:
                                    if step_child_level_4.tag == "DescriptorName":
                                        article_MeSH_list.append(step_child_level_4.text)
                                
                                article_MeSH = " ;".join(str(x) for x in article_MeSH_list)  
                        MeSH_list.append(article_MeSH)
                        
                 
            if step_child.tag == "PubmedData":
                for step_child_level_2 in step_child:
                    if step_child_level_2.tag == "ArticleIdList":
                        for step_child_level_3 in step_child_level_2:
                            if (step_child_level_3.tag == "ArticleId"):
                                if (step_child_level_3.attrib['IdType'] == 'pmc'):
                                    PMCID = step_child_level_3.text
                                else:
                                    PMCID = "None"
                PMCID_list.append(PMCID)
                                
                        
print (len(PMID_list))
print (len(ArtcileTitle_list))
print (len(Abstract_list))
print (len(PublicationType_list))
print (len(PMCID_list))

df = DataFrame({'PMID': PMID_list , 'Article Title': ArtcileTitle_list, 'Abstract': Abstract_list, 
                'Publication Type': PublicationType_list,'PMCID':PMCID_list })
df = df[['PMID','Article Title', 'Abstract', 'Publication Type','PMCID']]

#CHANGE THIS PATH TO THE FOLDER WHERE YOU STORE THE DATA
df.to_excel('/Data/All_Papers_Metadata.xlsx', sheet_name='sheet1', index=False)

8646822
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=8646822&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
8453848
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=8453848&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
23210726
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=23210726&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
11936570
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=11936570&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
1387301
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=1387301&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
8911886
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=8911886&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
14667956
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=14667956&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
1730173

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=17192770&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
16198653
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=16198653&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
9049584
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=9049584&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
17687273
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=17687273&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
21825288
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=21825288&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
11084219
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=11084219&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
16003291
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=16003291&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
18520949
h

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=11873000&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
19444001
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=19444001&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
8243504
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=8243504&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
6152055
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=6152055&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
15601809
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=15601809&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
9378846
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=9378846&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
8157044
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=8157044&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
16680561
https:/

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=9602962&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
9797793
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=9797793&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
10741630
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=10741630&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
19552747
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=19552747&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
3428348
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=3428348&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
18832427
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=18832427&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
10771455
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=10771455&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y
18520598
http